In [ ]:
%matplotlib inline
import numpy as np
from sklearn.model_selection import train_test_split
import time
import sys
import os
import argparse
from numpy.random import normal, uniform
from numpy.linalg import norm
import itertools
import pandas as pd
from matplotlib import pyplot as plt
import math
from sklearn.datasets import load_svmlight_file
import datetime
from IPython import display
from tqdm import tqdm
from enum import Enum

from contextlib import redirect_stdout
import shutil
import subprocess
from knuth_tournament.tournament import *
from quick_select.quickselect import *

In [ ]:
class AlgorithmType(Enum):
        TOURNAMENT = "tournament"
        QUICKSELECT = "quickselect"

# Test drive for the algorithms

## Tournament method

In [ ]:
def top_k_compressor(x, k):
    output = np.zeros(x.shape)
    x_abs = np.abs(x)
    idx = np.argpartition(x_abs, -k)[-k:]  # Indices not sorted
    inds = idx[np.argsort(x_abs[idx])][::-1]
    output[inds] = x[inds]
    return output

In [ ]:
input = [2, 16, 5, 12, -14, 8, -17, 10]
tournament = TournamentTopK()

k = 4
topk_knuth, numberOfComparisons = tournament.getTopK(input, k)
topk_test = top_k_compressor (np.array(input), k)

print("Top {} (knuth): {}".format(k, topk_knuth))
print("Top {} (my): {}".format(k, topk_test))
print("Total number Of comparisons:", numberOfComparisons)
print (np.allclose(topk_knuth, topk_test))

In [ ]:
topK, numberOfComparisons = tournament.getTopK(input, k)

## Quickselect

In [ ]:
myQuickselect = Quickselect()    
arr = np.random.rand(100000)
top, numberOfComparisons = myQuickselect.getTopK(arr, 5, pivotType=PivotType.RANDOM)
print("Random Pivot:\t\t", numberOfComparisons)

top, numberOfComparisons = myQuickselect.getTopK(arr, 5, pivotType=PivotType.DETERMINISTIC)
print("Deterministic Pivot:\t", numberOfComparisons)

top, numberOfComparisons = myQuickselect.getTopK(arr, 5, pivotType=PivotType.MEDIAN)
print("Median Pivot:\t\t", numberOfComparisons)

# Experiments on known prior

In [ ]:
def run_test(algorithm_type, pivot_type=None):
    project_path = os.getcwd() + "/"
    
    distribution = "normal"
    experiment_type = "synthetic"
    algorithm_name = algorithm_type.value
    data_path = project_path + "data_{0}/".format(experiment_type)
    if not os.path.exists(data_path):
        os.mkdir(data_path)

    #for real tests
    d_ar = np.array([10**3, 10**5, 10**7, 10**9], dtype=int)
    n_samples = 100_000

    #test values
    d_ar = np.array([10**2, 10**3, 10**4], dtype=int)
    #d_ar = np.array([10**2], dtype=int)
    n_samples = 100

    num_plot_points = 10

    if algorithm_type == AlgorithmType.TOURNAMENT:
        algorithm = TournamentTopK()
    else:
        algorithm = Quickselect()
        
    print("Selected algorithm is:", algorithm)

    experiment = '{0}_{1}'.format(experiment_type, algorithm_name, distribution)
    logs_path = project_path + "logs/logs_{0}/".format(experiment)

    # a folder to store precomputed sample matrices
    if not os.path.exists(data_path):
        os.makedirs(data_path)

    #a global folder to store time complexity results
    if not os.path.exists(project_path + "logs/"):
        os.makedirs(project_path + "logs/")

    #a folder to store time complexity results of the experiments groupped by experiment_type, algorithm_name and distribution
    if not os.path.exists(logs_path):
        os.makedirs(logs_path)

    for d in d_ar:
        #for each d create sample matrices 
        sm_name = 'SM_prior-{0}_n-{1}_d-{2}.npy'.format(distribution, n_samples, d)
        if not os.path.isfile(data_path + sm_name):
            #TODO: make this generation reproducible fix some seed value
            sample_matrix =  np.random.normal(loc=0.0, scale=1.0, size=(n_samples, d))
            np.save(data_path + sm_name, sample_matrix)
        else:
            sample_matrix = np.load(data_path + sm_name)

        k_ar = np.linspace (max(int(0.01*d),1), d, num_plot_points, dtype=int)
        tc_matrix = np.zeros((n_samples,k_ar.shape[0]))
        tc_median = np.zeros(k_ar.shape[0])

        for i_s in tqdm(range(n_samples)):
            for i_k in range(k_ar.shape[0]):
                if algorithm_type == AlgorithmType.TOURNAMENT:
                    topk_k, numberOfComparisons = algorithm.getTopK(list(sample_matrix[i_s]), k_ar[i_k])
                else:
                    topk_k, numberOfComparisons = algorithm.getTopK(list(sample_matrix[i_s]), k_ar[i_k], pivotType=pivot_type)
                topk_my = top_k_compressor (sample_matrix[i_s], k_ar[i_k])
                tc_matrix [i_s, i_k] = numberOfComparisons
                if not np.allclose(topk_k, topk_my):
                    print("Top {} (knuth): {}".format(k_ar[i_k], topk_k))
                    print("Top {} (my): {}".format(k_ar[i_k], topk_my))

        tcm_name = 'TCM_prior-{0}_n-{1}_d-{2}.npy'.format(distribution, n_samples, d)
        np.save(logs_path + tcm_name, tc_matrix)
        
    return algorithm, sample_matrix


## Tournament method

In [ ]:
algorithm, sample_matrix = run_test(AlgorithmType.TOURNAMENT)

In [ ]:
# logs_path + tcm_name

In [ ]:
#crashable input
i_s = 0
k = 9
tenMinimum, numberOfComparisons = algorithm.getTopK(list(sample_matrix[i_s]), k)

## Quickselect method

In [ ]:
algorithm, sample_matrix = run_test(AlgorithmType.QUICKSELECT, pivot_type=PivotType.RANDOM)
# algorithm, sample_matrix = run_test(AlgorithmType.QUICKSELECT, pivot_type=PivotType.DETERMINISTIC)
# algorithm, sample_matrix = run_test(AlgorithmType.QUICKSELECT, pivot_type=PivotType.MEDIAN)

# Auxillary section

In [ ]:

k_ar_slice = np.array([0.01*d, 0.1*d, 0.2*d, 0.5*d], dtype=int)


## Histogramm axis correctess check

In [ ]:
k_ar

In [ ]:
d_ar = np.array([10**3, 10**5, 10**7, 10**9], dtype=int)
n_samples = 100_000
num_plot_points = 100

for d in d_ar:
    k_ar = np.linspace ( int(0.01*d), d, num_plot_points)
    k_ar_slice = np.array([0.01*d, 0.1*d, 0.2*d, 0.5*d], dtype=int)
    print (all(k_s in k_ar for k_s in k_ar_slice))